In [2]:
from bs4 import BeautifulSoup
from datetime import datetime
from psycopg2 import sql, OperationalError
import logging
import numpy as np
import openpyxl
import os
import pandas as pd
import psycopg2
from psycopg2.extras import execute_values
import requests
from sendgrid import SendGridAPIClient
from sendgrid.helpers.mail import Mail, Content
import xlsxwriter

In [3]:
CURRENT_TIMESTAMP = datetime.now()
formated_current_timestamp = CURRENT_TIMESTAMP.strftime("%Y%m%d_%H%M%S")
list_with_offers = []
seen_offers = set()

In [4]:
# FUNC FOR RETRIEVING THE NUMBER OF PAGES TO SCRAPE
def func_get_num_of_pages(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, "html.parser")
        num_of_pages = int(soup.find_all("a", class_="page-link")[-2].text) + 1
        logging.info(f"Number of pages to scrape: {num_of_pages - 1}")
        return num_of_pages
    except requests.RequestException as e:
        logging.error(f"Error fetching the number of pages: {e}")
        return 0

# SELECT FROM HTML OFFERS ONLY BLOCKS OF CODE WHICH OUR INTERESTING FOR US FROM THE ENTIRE HTML PAGE
def func_get_offers_from_page(page_url):
    try:
        response = requests.get(page_url)
        soup = BeautifulSoup(response.content, "html.parser")
        offers_block = soup.find_all("article", "PropertyCard_propertyCard__moO_5 propertyCard PropertyCard_propertyCard--landscape__XvPmC")
        return offers_block
    except requests.RequestException as e:
        logging.error(f"Error fetching the offer links: {e}")
        return []

# FUNC FOR EXTRACTING DETAILS FROM EACH OFFER
def func_extract_offer_details(offer, page):
    try:
        flat_price = offer.find("span", class_="PropertyPrice_propertyPriceAmount__WdEE1").text
        flat_og_price = offer.find("span", class_="PropertyPrice_propertyPriceOriginal__6POAT").text if offer.find("span", class_="PropertyPrice_propertyPriceOriginal__6POAT") else np.nan
        flat_fees = offer.find("span", class_="PropertyPrice_propertyPriceAdditional__5jYQ6").text if offer.find("span", class_="PropertyPrice_propertyPriceAdditional__5jYQ6") else np.nan
        flat_info = offer.find("p", class_="mt-2 mt-md-3 mb-0 text-caption text-truncate-multiple").text
        features_list = offer.find_all("li", class_="FeaturesList_featuresListItem__RYf_f")
        flat_disposition = features_list[0].text if len(features_list) > 0 else np.nan
        flat_size = features_list[1].text if len(features_list) > 1 else np.nan
        flat_title = offer.find("span", class_="PropertyCard_propertyCardAddress__hNqyR text-subheadline text-truncate").text
        flat_href = offer.find("a").get("href")
        
        return {
            "page": page,
            "title": flat_title,
            "href": flat_href,
            "price": flat_price,
            "og_price": flat_og_price,
            "fee": flat_fees,
            "disposition": flat_disposition,
            "size": flat_size,
            "offer_info": flat_info,
            "scrape_timestamp": CURRENT_TIMESTAMP
        }
    except AttributeError as e:
        logging.error(f"Error extracting offer details: {e}")
        return {}

def func_scrape_page(page_url, page_num):
    offers_block = func_get_offers_from_page(page_url)
    for offer in offers_block:
        offer_details = func_extract_offer_details(offer, page_num)
        if offer_details:
            # Check if the offer is unique using the URL
            if offer_details["href"] not in seen_offers:
                seen_offers.add(offer_details["href"])
                list_with_offers.append(offer_details)

def func_extract():
    base_url = "https://www.bezrealitky.cz/vyhledat?offerType=PRONAJEM&estateType=BYT&regionOsmIds=R435541&osm_value=Praha%2C+%C4%8Cesko&location=exact&currency=CZK&page="
    num_of_pages_url = base_url + "1"
    num_of_pages = func_get_num_of_pages(num_of_pages_url)

    for page in range(1, num_of_pages):
        page_url = base_url + str(page)
        func_scrape_page(page_url, page)
        print(f"Scraped page: {page}")
    print(f"Offers scraped: {len(list_with_offers)}")

In [5]:
# CREATE TABLE IF NOT ALREADY CREATED
def func_initialize_connection():
    try:
        conn = psycopg2.connect(
            host="localhost",
            database="bezrealitky",
            user="postgres",
            password="postgres"
        )
        return conn
    except OperationalError as e:
        print(f"Error connecting to the database: {e}")
        return None

# CRATING VALUE FOR DAILY BATCH NUM BASED ON MAX DAILY_BATCH_NUM
def func_daily_batch_num():
    conn = func_initialize_connection()
    if conn is None:
        return None

    try:
        with conn:
            with conn.cursor() as curr:
                current_day = datetime.now().date()
                curr.execute("SELECT COALESCE(MAX(daily_batch_num), 0) FROM etl_rent WHERE scrape_date = %s;", (current_day,))
                result = curr.fetchone()
                num_of_actual_batches = result[0]
                total_batch_id = num_of_actual_batches + 1
    except Exception as e:
        print(f"Error executing the query: {e}")
        return None
    finally:
        conn.close()

    return total_batch_id

# CRATING VALUE FOR DAILY BATCH NUM BASED ON TOTAL_BATCH_NUM
def func_total_batch_num():
    conn = func_initialize_connection()
    if conn is None:
        return None

    try:
        with conn:
            with conn.cursor() as curr:
                curr.execute("SELECT COALESCE(MAX(total_batch_num), 0) FROM etl_rent;")
                result = curr.fetchone()
                num_of_actual_batches = result[0]
                total_batch_id = num_of_actual_batches + 1
    except Exception as e:
        print(f"Error executing the query: {e}")
        return None
    finally:
        conn.close()

    return total_batch_id

# CREATE ETL_RENT TABLE IF NOT EXISTS
def func_create_table():
    conn = func_initialize_connection()
    if conn is None:
        return

    try:
        with conn:
            with conn.cursor() as curr:
                curr.execute("""
                    CREATE TABLE IF NOT EXISTS etl_rent (
                        id_offer INT,
                        scrape_timestamp TIMESTAMP,
                        scrape_time TIME,
                        scrape_date DATE,
                        scrape_weekday INT,
                        total_batch_num INT,
                        daily_batch_num INT,
                        price_czk FLOAT,
                        og_price_czk FLOAT,
                        discount FLOAT,
                        total_price FLOAT,
                        fee_czk FLOAT,
                        total_price_per_m2_czk FLOAT,
                        disposition TEXT,
                        size_m2 FLOAT,
                        street TEXT,
                        district TEXT,
                        prague_region TEXT,
                        z_score_total_price_per_m2_czk FLOAT,
                        mhd_minutes_away FLOAT,
                        balcony_size_m2 FLOAT,
                        cellar_size_m2 FLOAT,
                        terrace_size_m2 FLOAT,
                        parking_or_and_garage TEXT,
                        elevator TEXT,
                        partly_or_fully_equiped TEXT,
                        page INT,
                        href TEXT
                    );
                """)
                conn.commit()
    except Exception as e:
        print(f"Error creating the table: {e}")
    finally:
        conn.close()

In [6]:
def func_transformation():
    # CREATE A PANDAS DATAFRAME
    df = pd.DataFrame(list_with_offers)
    df["scrape_timestamp"] = pd.to_datetime(df["scrape_timestamp"])

    # REPLACE NAN VALUES WITH NONE VALUES
    df = df.replace({np.nan: None})
    
    # DROP DUPLICATES
    df.drop_duplicates(subset="href", inplace=True)
    
    # CONVERT MHD TO LOWERCASE FOR BETTER MANIPULATION
    df["offer_info"] = df["offer_info"].str.lower()
    
    # DISTRICT AND STREET COLUMN CREATION (DIFFERENT APPROACH FOR STŘEDOČESKÝ KRAJ)
    df["district"] = df.apply(lambda row: row["title"].split(",")[-2].split("-")[0].lstrip(" ").rstrip(" ") if "Středočeský kraj" in row["title"] else row["title"].split("Praha")[-1].split("-")[1].strip() if len(row["title"].split("Praha")) > 1 else "", axis=1)
    df["street"] = df.apply(lambda row: row["title"].split(",")[0] if "Středočeský kraj" in row["title"] else row["title"].split("Praha")[0].replace(",", "").rstrip(" ") if len(row["title"].split("Praha")) > 1 else row["title"], axis=1)

    # PRAGUE REGION COLUMN CREATION
    df["prague_region"] = df['title'].str.contains('praha', case=False).map({True: 'Y', False: 'N'})
    
    # PRICE COLUMN TRANSFORMATION
    df["price"] = df['price'].str.extract(r'(\d[\d\s]*)')
    df["price"] = df["price"].str.replace("\xa0", "")
    df["price"] = df["price"].str.replace(" ", "")
    df["price"] = df["price"].astype("float64")
    df.rename(columns={"price":"price_czk"}, inplace=True)
    
    # OG PRICE COLUMN TRANSFORMATION
    df["og_price"] = df['og_price'].str.extract(r'(\d[\d\s]*)')
    df["og_price"] = df["og_price"].str.replace("\xa0", "")
    df["og_price"] = df["og_price"].str.replace(" ", "")
    df["og_price"] = df["og_price"].astype("float64")
    df.rename(columns={"og_price":"og_price_czk"}, inplace=True)

    # FEE COLUMN CREATION
    df["fee"] = df['fee'].str.extract(r'(\d[\d\s]*)')
    df["fee"] = df["fee"].str.replace("\xa0", "")
    df["fee"] = df["fee"].str.replace(" ", "")
    df["fee"] = df["fee"].astype("float64")
    df.rename(columns={"fee":"fee_czk"}, inplace=True)

    # TOTAL PRICE COLUMN CREATION
    df["total_price"] = df["price_czk"]+df["fee_czk"].fillna(0)
    
    # DISCOUNT COLUMN CREATION
    df["discount"] = df["og_price_czk"]-df["price_czk"]
    
    # SIZE TRANSFORMATION
    df["size"] = df["size"].str.replace("m²", "")
    df["size"] = df["size"].astype("float64")
    df.rename(columns={"size":"size_m2"}, inplace=True)
    
    # SIZE OF AMENITIES
    def func_text_after_func(column, key_word):
        temp_text_after = df[column].str.extract(rf'{key_word}[:\s]*(.*?)(?:\s+-|$)').reset_index()
        temp_text_after = temp_text_after.iloc[:, 1].str.split(" ", n=1).str.get(0)
        return temp_text_after

    # TRANSPORT MINUTES AWAY COLUMN CREATION
    df["mhd_minutes_away"] = func_text_after_func("offer_info", "mhd")
    df.loc[df["mhd_minutes_away"] == "do", "mhd_minutes_away"] = 1

    # BALCONY SIZE COLUMN CREATION
    df["balcony_size_m2"] = func_text_after_func("offer_info", "balkón")
    df["balcony_size_m2"] = df["balcony_size_m2"].str.replace(",", ".")

    # CELLAR SIZE COLUMN CREATION
    df["cellar_size_m2"] = func_text_after_func("offer_info", "sklep")
    df["cellar_size_m2"] = df["cellar_size_m2"].str.replace(",", ".")

    # TERRACE SIZE COLUMN CREATION
    df["terrace_size_m2"] = func_text_after_func("offer_info", "terasa")
    df["terrace_size_m2"] = df["terrace_size_m2"].str.replace(",", ".")
    
    # Y OR N FLAG FOR UTILITIES
    def func_text_contains_flag_func(column, keyword):
        temp_flag_outcome = df[column].apply(lambda x: "Y" if keyword in x.lower() else "N")
        return temp_flag_outcome

    # GARAGE OR PARKING SPACE COLUMN CREATION
    df["parking_or_and_garage"] = df["offer_info"].apply(lambda x: "Y" if "garáž" in x.lower() or "parkování" in x.lower() else "N")

    # ELEVATOR COLUMN CREATION
    df["elevator"] = func_text_contains_flag_func("offer_info", "výtah")

    # EQUIPED FLAT COLUMN CREATION
    df["partly_or_fully_equiped"] = func_text_contains_flag_func("offer_info", "vybaveno")
    
    # CREATE DAY, TIME, WEEKDAY COLUMNS
    df["scrape_weekday"] = df["scrape_timestamp"].dt.day_of_week+1 # MONDAY 0->1
    df["scrape_time"] = df["scrape_timestamp"].dt.time
    df["scrape_date"] = df["scrape_timestamp"].dt.date
    
    # CREATING TOTAL_BATCH_NUM A DAILY_BATCH_NUM COLUMNS
    df["total_batch_num"] = func_total_batch_num()
    df["daily_batch_num"] = func_daily_batch_num()

    # CREATE ID COLUMN
    df["id_offer"] = df["href"].str.split("domy/").str.get(1).str.split("-").str.get(0)

    # CREATE PRICE PER M2 COLUMN
    df["total_price_per_m2_czk"] = (df["total_price"]/df["size_m2"]).round(2)

    # Z-SCORE COLUMN CREATION
    df["z_score_total_price_per_m2_czk"] = (df["total_price_per_m2_czk"] - df["total_price_per_m2_czk"].mean()) / df["total_price_per_m2_czk"].std()
    
    # DF COLUMNS EDIT
    df.drop(columns=["title", "offer_info"], inplace=True)
    df = df.replace({"": None})


    df = df[
    [
    "id_offer",
    "scrape_timestamp",
    "scrape_time",
    "scrape_date",
    "scrape_weekday",
    "total_batch_num",
    "daily_batch_num",
    "price_czk",
    "og_price_czk",
    "discount",
    "total_price",
    "fee_czk",
    "total_price_per_m2_czk",
    "disposition",
    "size_m2",
    "street",
    "district",
    "prague_region",
    "z_score_total_price_per_m2_czk",
    "mhd_minutes_away",
    "balcony_size_m2",
    "cellar_size_m2",
    "terrace_size_m2",
    "parking_or_and_garage",
    "elevator",
    "partly_or_fully_equiped",
    "page",
    "href"
    ]]

    print("Data transformed")

    # CREATE TABLE IF NOT EXISTS 
    func_create_table()
    
    return df

In [7]:
# LOAD OF DATA TO DATABASE
def func_load():
    conn = conn = psycopg2.connect(
        host="localhost",
        database="bezrealitky",
        user="postgres",
        password="postgres"
    )
    curr = conn.cursor()

    df_to_tuple_values = [tuple(x) for x in df.to_numpy()]
    table_columns = ','.join(list(df.columns))
    insert_query = f"INSERT INTO etl_rent ({table_columns}) VALUES %s"

    execute_values(curr, insert_query, df_to_tuple_values)
    conn.commit()

    curr.close()
    conn.close()
    print(f"{df.shape[0]} rows inserted")

In [8]:
# PULLING DATA FROM DATABASE VIEW (MAIN VIEW)
def func_view_conversion():
    conn = func_initialize_connection()
    if conn is None:
        return None

    try:
        with conn:
            with conn.cursor() as curr:
                curr.execute("""
                                WITH main AS (
                                    SELECT
                                        id_offer,
                                        MIN(scrape_timestamp) min_timestamp,
                                        CASE
                                            WHEN MIN(scrape_timestamp) = '2024-06-21 22:59:15.834483'
                                            THEN 'Y'
                                            ELSE 'N'
                                        END first_load,
                                        MAX(scrape_timestamp) max_timestamp,
                                        ROUND(EXTRACT(EPOCH FROM (MAX(scrape_timestamp) - MIN(scrape_timestamp))) / 3600/24, 2) time_on_market,
                                        CASE
                                            WHEN MAX(scrape_timestamp) = (SELECT MAX(scrape_timestamp) FROM etl_rent)
                                            THEN 'Y'
                                            ELSE 'N'
                                        END active_offer,
                                        MAX(total_price) max_total_price,
                                        MIN(total_price) min_total_price,
                                        ROUND(CAST((1 - (MIN(total_price) / MAX(total_price))) * 100 AS numeric), 2) min_max_total_price_diff
                                    FROM etl_rent
                                    GROUP BY id_offer
                                )

                                ,latest_info AS (
                                SELECT
                                    latest.id_offer,
                                    latest.price_czk,
                                    latest.fee_czk,
                                    latest.total_price,
                                    latest.disposition,
                                    disp_price.avg_total_price_per_disposition,
                                    ROUND(CAST((latest.total_price - disp_price.avg_total_price_per_disposition) / disp_price.avg_total_price_per_disposition * 100 AS numeric), 2) AS total_price_per_disp_diff,
                                    latest.district,
                                    latest.size_m2,
                                    latest.total_price_per_m2_czk,
                                    dist_price.avg_total_price_m2_per_district,
                                    ROUND(CAST((latest.total_price_per_m2_czk - dist_price.avg_total_price_m2_per_district) / dist_price.avg_total_price_m2_per_district * 100 AS numeric), 2) AS total_price_m2_per_district_diff,
                                    latest.z_score_total_price_per_m2_czk,
                                    latest.street,
                                    latest.prague_region,
                                    latest.mhd_minutes_away,
                                    latest.balcony_size_m2,
                                    latest.cellar_size_m2,
                                    latest.terrace_size_m2,
                                    latest.parking_or_and_garage,
                                    latest.elevator,
                                    latest.partly_or_fully_equiped,
                                    latest.href
                                FROM (
                                    SELECT * FROM (
                                        SELECT
                                            *,
                                            ROW_NUMBER() OVER (PARTITION BY id_offer ORDER BY scrape_timestamp DESC) rn
                                        FROM etl_rent
                                    )
                                    WHERE rn = 1
                                ) latest
                                JOIN (
                                    SELECT disposition, AVG(total_price) avg_total_price_per_disposition
                                    FROM etl_rent
                                    WHERE True
                                        AND total_batch_num = (SELECT MAX(total_batch_num) FROM etl_rent)
                                        AND z_score_total_price_per_m2_czk < 3
                                    GROUP BY disposition
                                ) disp_price
                                    ON latest.disposition = disp_price.disposition
                                LEFT JOIN (
                                    SELECT district, AVG(total_price_per_m2_czk) avg_total_price_m2_per_district
                                    FROM etl_rent
                                    WHERE True
                                        AND total_batch_num = (SELECT MAX(total_batch_num) FROM etl_rent)
                                        AND z_score_total_price_per_m2_czk < 3
                                    GROUP BY district
                                ) dist_price
                                    ON latest.district = dist_price.district
                                )

                                SELECT
                                    main.id_offer,
                                    main.min_timestamp,
                                    main.first_load,
                                    main.max_timestamp,
                                    main.time_on_market,
                                    main.active_offer,
                                    main.min_total_price,
                                    main.max_total_price,
                                    main.min_max_total_price_diff,
                                    latest_info.price_czk current_price,
                                    latest_info.fee_czk current_fee,
                                    latest_info.total_price current_total_price,
                                    latest_info.total_price_per_m2_czk total_price_per_m2,
                                    latest_info.district,
                                    area_proximity.district_proximity,
                                    latest_info.disposition,
                                    latest_info.size_m2,
                                    latest_info.total_price_per_disp_diff,
                                    latest_info.total_price_m2_per_district_diff,
                                    latest_info.z_score_total_price_per_m2_czk z_score_total_price_per_m2,
                                    latest_info.street,
                                    latest_info.prague_region,
                                    latest_info.mhd_minutes_away,
                                    latest_info.balcony_size_m2,
                                    latest_info.cellar_size_m2,
                                    latest_info.terrace_size_m2,
                                    latest_info.parking_or_and_garage,
                                    latest_info.elevator,
                                    latest_info.partly_or_fully_equiped,
                                    latest_info.href
                                FROM main
                                JOIN latest_info
                                    ON main.id_offer = latest_info.id_offer
                                LEFT JOIN area_proximity
                                    ON LOWER(latest_info.district) = LOWER(area_proximity.district)
                                ;""")
                rows = curr.fetchall()
                column_names = [desc[0] for desc in curr.description]
                df_view_postgres = pd.DataFrame(rows, columns=column_names)
                
                return df_view_postgres
    
    except Exception as e:
        print(f"Error executing the query: {e}")
        return None
    finally:
        conn.close()

In [9]:
# CREATING EXCEL FILE AND THEN EXPORTING THE FILE TO THE SHARED FOLDER
def func_excel_export():
    df_view = func_view_conversion()
    
    # FORMATTING COLUMNS FOR EXCEL (COMMAS INSTEAD OF FULLSTOPS)
    df_view["total_price_per_disp_diff"] = df_view["total_price_per_disp_diff"].astype("float64")
    df_view["total_price_m2_per_district_diff"] = df_view["total_price_m2_per_district_diff"].astype("float64")
    df_view["time_on_market"] = df_view["time_on_market"].astype("float64")
    df_view["min_max_total_price_diff"] = df_view["min_max_total_price_diff"].astype("float64")
    df_view["z_score_total_price_per_m2"] = df_view["z_score_total_price_per_m2"].astype("float64")

    # NAME + PATH FOR EXPORT
    path = r"C:\path\to\your\export\directory"
    name = f"{formated_current_timestamp}_etl_rent.xlsx"
    filename = os.path.join(path, name)

    # EXCEL FILE CREATION
    with pd.ExcelWriter(filename, engine='xlsxwriter') as writer:
        df_view.to_excel(writer, index=False, sheet_name='Sheet1')
        workbook  = writer.book
        worksheet = writer.sheets['Sheet1']
        
        # DEFINING TABLE RANGE
        (max_row, max_col) = df_view.shape
        column_settings = [{'header': column} for column in df_view.columns]
        
        # ADDING TABLE
        worksheet.add_table(0, 0, max_row, max_col - 1, {'columns': column_settings, 'name': 'Table1', 'style': 'Table Style Medium 9'})
        
        # COLUMN WIDTH ADJUSTMENT
        for i, col in enumerate(df_view.columns):
            column_len = df_view[col].astype(str).str.len().max()
            column_len = max(column_len, len(col)) + 2
            worksheet.set_column(i, i, column_len)
        
        # HREF COLUMN HYPERLINK CREATION
        hyperlink_format = workbook.add_format({'color': 'blue', 'underline': 1})
        for row_num, link in enumerate(df_view['href'], start=1):
            worksheet.write_url(row_num, df_view.columns.get_loc('href'), link, hyperlink_format, string=link)
        
        # CONDITIONAL FORMATTING FOR total_price_per_disp_diff AND total_price_m2_per_district_diff (RED AND GREEN COLOR)
        columns_to_format = ['total_price_per_disp_diff', 'total_price_m2_per_district_diff']
        for column in columns_to_format:
            col_index = df_view.columns.get_loc(column)
            worksheet.conditional_format(1, col_index, max_row, col_index, {'type': 'cell', 'criteria': '<', 'value': 0, 'format': workbook.add_format({'bg_color': '#C6EFCE', 'font_color': '#006100'})})
            worksheet.conditional_format(1, col_index, max_row, col_index, {'type': 'cell', 'criteria': '>', 'value': 0, 'format': workbook.add_format({'bg_color': '#FFC7CE', 'font_color': '#9C0006'})})
    
    print("File exported")

In [10]:
# FUNC FOR VIEW WHICH CHECKS PRICE DIFFERENCE BETWEEN LOADS -> WILL BE USED AS HTML IN EMAIL BODY
def func_view_price_diff_conversion():
    conn = func_initialize_connection()
    if conn is None:
        return None
    try:
        with conn:
            with conn.cursor() as curr:
                curr.execute("""
                            WITH lag_price AS (
                            SELECT
                                id_offer,
                                scrape_timestamp previous_timestamp,
                                total_batch_num,
                                total_price current_total_price,
                                LAG(scrape_timestamp) OVER (PARTITION BY id_offer ORDER BY scrape_timestamp DESC) current_timestamp,
                                LAG(total_price) OVER (PARTITION BY id_offer ORDER BY scrape_timestamp DESC) previous_total_price,
                                ROW_NUMBER() OVER (PARTITION BY id_offer ORDER BY scrape_timestamp DESC) rn
                            FROM etl_rent
                            ORDER BY id_offer, scrape_timestamp DESC
                            )
                            ,previous_load_price_diff AS (
                            SELECT
                                id_offer,
                                current_timestamp,
                                previous_total_price,
                                previous_timestamp,
                                current_total_price,
                                (1-(current_total_price/previous_total_price))*100 previous_load_price_diff
                            FROM lag_price
                            WHERE True
                                AND rn = 2
                                AND current_total_price < previous_total_price
                                AND (1-(current_total_price/previous_total_price))*100 > 1
                            )

                            SELECT
                                prev_load.id_offer,
                                prev_load.previous_timestamp,
                                prev_load.previous_total_price,
                                prev_load.current_timestamp,
                                prev_load.current_total_price,
                                prev_load.previous_load_price_diff discount_pct,
                                v_etl_rent.active_offer,
                                v_etl_rent.total_price_per_m2,
                                v_etl_rent.disposition,
                                v_etl_rent.district,
                                v_etl_rent.href
                            FROM previous_load_price_diff prev_load
                            JOIN (WITH main AS (
                                    SELECT
                                        id_offer,
                                        MIN(scrape_timestamp) min_timestamp,
                                        CASE
                                            WHEN MIN(scrape_timestamp) = '2024-06-21 22:59:15.834483'
                                            THEN 'Y'
                                            ELSE 'N'
                                        END first_load,
                                        MAX(scrape_timestamp) max_timestamp,
                                        ROUND(EXTRACT(EPOCH FROM (MAX(scrape_timestamp) - MIN(scrape_timestamp))) / 3600/24, 2) time_on_market,
                                        CASE
                                            WHEN MAX(scrape_timestamp) = (SELECT MAX(scrape_timestamp) FROM etl_rent)
                                            THEN 'Y'
                                            ELSE 'N'
                                        END active_offer,
                                        MAX(total_price) max_total_price,
                                        MIN(total_price) min_total_price,
                                        ROUND(CAST((1 - (MIN(total_price) / MAX(total_price))) * 100 AS numeric), 2) min_max_total_price_diff
                                    FROM etl_rent
                                    GROUP BY id_offer
                                )

                                ,latest_info AS (
                                SELECT
                                    latest.id_offer,
                                    latest.price_czk,
                                    latest.fee_czk,
                                    latest.total_price,
                                    latest.disposition,
                                    disp_price.avg_total_price_per_disposition,
                                    ROUND(CAST((latest.total_price - disp_price.avg_total_price_per_disposition) / disp_price.avg_total_price_per_disposition * 100 AS numeric), 2) AS total_price_per_disp_diff,
                                    latest.district,
                                    latest.size_m2,
                                    latest.total_price_per_m2_czk,
                                    dist_price.avg_total_price_m2_per_district,
                                    ROUND(CAST((latest.total_price_per_m2_czk - dist_price.avg_total_price_m2_per_district) / dist_price.avg_total_price_m2_per_district * 100 AS numeric), 2) AS total_price_m2_per_district_diff,
                                    latest.z_score_total_price_per_m2_czk,
                                    latest.street,
                                    latest.prague_region,
                                    latest.mhd_minutes_away,
                                    latest.balcony_size_m2,
                                    latest.cellar_size_m2,
                                    latest.terrace_size_m2,
                                    latest.parking_or_and_garage,
                                    latest.elevator,
                                    latest.partly_or_fully_equiped,
                                    latest.href
                                FROM (
                                    SELECT * FROM (
                                        SELECT
                                            *,
                                            ROW_NUMBER() OVER (PARTITION BY id_offer ORDER BY scrape_timestamp DESC) rn
                                        FROM etl_rent
                                    )
                                    WHERE rn = 1
                                ) latest
                                JOIN (
                                    SELECT disposition, AVG(total_price) avg_total_price_per_disposition
                                    FROM etl_rent
                                    WHERE True
                                        AND total_batch_num = (SELECT MAX(total_batch_num) FROM etl_rent)
                                        AND z_score_total_price_per_m2_czk < 3
                                    GROUP BY disposition
                                ) disp_price
                                    ON latest.disposition = disp_price.disposition
                                LEFT JOIN (
                                    SELECT district, AVG(total_price_per_m2_czk) avg_total_price_m2_per_district
                                    FROM etl_rent
                                    WHERE True
                                        AND total_batch_num = (SELECT MAX(total_batch_num) FROM etl_rent)
                                        AND z_score_total_price_per_m2_czk < 3
                                    GROUP BY district
                                ) dist_price
                                    ON latest.district = dist_price.district
                                )

                                SELECT
                                    main.id_offer,
                                    main.min_timestamp,
                                    main.first_load,
                                    main.max_timestamp,
                                    main.time_on_market,
                                    main.active_offer,
                                    main.min_total_price,
                                    main.max_total_price,
                                    main.min_max_total_price_diff,
                                    latest_info.price_czk current_price,
                                    latest_info.fee_czk current_fee,
                                    latest_info.total_price current_total_price,
                                    latest_info.total_price_per_m2_czk total_price_per_m2,
                                    latest_info.district,
                                    area_proximity.district_proximity,
                                    latest_info.disposition,
                                    latest_info.size_m2,
                                    latest_info.total_price_per_disp_diff,
                                    latest_info.total_price_m2_per_district_diff,
                                    latest_info.z_score_total_price_per_m2_czk z_score_total_price_per_m2,
                                    latest_info.street,
                                    latest_info.prague_region,
                                    latest_info.mhd_minutes_away,
                                    latest_info.balcony_size_m2,
                                    latest_info.cellar_size_m2,
                                    latest_info.terrace_size_m2,
                                    latest_info.parking_or_and_garage,
                                    latest_info.elevator,
                                    latest_info.partly_or_fully_equiped,
                                    latest_info.href
                                FROM main
                                JOIN latest_info
                                    ON main.id_offer = latest_info.id_offer
                                LEFT JOIN area_proximity
                                    ON LOWER(latest_info.district) = LOWER(area_proximity.district)
                             ) v_etl_rent
                                ON prev_load.id_offer = v_etl_rent.id_offer
                                AND active_offer = 'Y';
                            """)
                rows = curr.fetchall()
                column_names = [desc[0] for desc in curr.description]
                df_view_price_diff_postgres = pd.DataFrame(rows, columns=column_names)
                
                return df_view_price_diff_postgres
    
    except Exception as e:
        print(f"Error executing the query: {e}")
        return None
    finally:
        conn.close()

In [11]:
# FUNC FOR SENDING EMAIL NOTIFICATION VIA SENDGRID WITH ONEDRIVE LINK
def func_send_email_notification():
    api_key = "SENDGRID_API_KEY"
    sender = "your_email@example.com"
    reciever = ["recipient1@example.com", "recipient2@example.com"]
    subject = f"{formated_current_timestamp} - nabídky bytů Praha"
    onedrive_link = "ONEDRIVE_LINK"

    # DF CONVERSION TO HTML
    df_price_diff_view = func_view_price_diff_conversion()
    html_table = df_price_diff_view.to_html(index=False, border=0)

    # HTML EMAIL CONTENT
    html_content = f"""
    <!DOCTYPE html>
    <html>
    <head>
        <meta charset="UTF-8">
        <style>
            table {{
                width: 50%;
                border-collapse: collapse;
            }}
            table, th, td {{
                border: 1px solid black;
                padding: 5px;
                text-align: left;
            }}
            th, td {{
                padding: 10px;
            }}
        </style>
    </head>
    <body>
        <p>Ahoj! Rád bych vás informoval, že proběhl nový load dat.</p>
        <p>Sdílená onedrive sloužka se soubory: <a href="{onedrive_link}">Onedrive link</a></p>
        <p>V případě, že nabídka má od minulého loadu nižší cenu, bude vidět v tabulce níže:</p>
        {html_table}
        <p>Ať slouží! Kilián</p>
    </body>
    </html>
    """

    # EMAIL CREATION
    email = Mail(
        from_email=sender,
        to_emails=reciever,
        subject=subject,
        html_content=html_content
    )

    try:
        # SEND EMAIL
        sg = SendGridAPIClient(api_key)
        response = sg.send(email)
        print(f"Status code: {response.status_code}")
        print("Email sent")

    except Exception as e:
        print(f"Error: {e}")

In [12]:
if __name__ == "__main__":
    func_extract()
    df = func_transformation()
    df = df.replace({np.nan: None})
    func_load()
    func_excel_export()
    func_send_email_notification()